**Merge_geojsons_into_one**
- For converting qs into cbe-air, map is rendered using geojson rendering capability of github
- the marker was easiliy made into geojson from qgis and org2ogr as a shape file. based on [this](https://github.com/cambecc/earth) under
  section "getting map data", the command is
```bash
ogr2ogr -f GeoJSON point.json point.shp
//and adding this script line in html
<script src="https://embed.github.com/view/geojson/saconswl/cbeair/gh-pages/cbe-s.json?height=530&width=1300"></script>
//under div map
```
- To include coimbatore city limits along with point marker, adding another script line of github embed renderes another map!
- so the solution is to merge point shape file and polygon shape file, since shape file doesn't support different feature(pooint.polygon) mergeing
- it has to done it in level of ogr or geojosn level. [guide from this](http://gis.stackexchange.com/questions/74627/can-we-merge-shapefiles-with-different-geometries-point-and-polygon-to-get-a-s)
- to point the [solution](http://gis.stackexchange.com/questions/74175/ogr-merge-shapefiles-of-different-geom-type-into-a-single-geojson-feature-collec?rq=1)
- in which small commant gives a [python code](https://gist.github.com/migurski/3759608) to merge two geojson file into one
- Using that python script, copied bellow from https://gist.github.com/migurski/3759608 merged point geojosn and polytgon geojson into one. 
```bash
python merge-geojsons.py cbe_city.json dylos_sta.json cbe-s.json
```
- the code become 
```python
from json import load, JSONEncoder
from optparse import OptionParser
from re import compile
float_pat = compile(r'^-?\d+\.\d+(e-?\d+)?$')
charfloat_pat = compile(r'^[\[,\,]-?\d+\.\d+(e-?\d+)?$')
parser = OptionParser(usage="""%prog [options]
#Group multiple GeoJSON files into one output file.
#Example:
#python %prog -p 2 input-1.json input-2.json output.json""")
defaults = dict(precision=6)
parser.set_defaults(**defaults)
parser.add_option('-p', '--precision', dest='precision',
                  type='int', help='Digits of precision, default %(precision)d.' % defaults)
if __name__ == '__main__':
    options, args = parser.parse_args()
    infiles, outfile = args[:-1], args[-1]
    outjson = dict(type='FeatureCollection', features=[])
    for infile in infiles:
        injson = load(open(infile))
        if injson.get('type', None) != 'FeatureCollection':
            raise Exception('Sorry, "%s" does not look like GeoJSON' % infile)
        if type(injson.get('features', None)) != list:
            raise Exception('Sorry, "%s" does not look like GeoJSON' % infile)
        outjson['features'] += injson['features']
    encoder = JSONEncoder(separators=(',', ':'))
    encoded = encoder.iterencode(outjson)
    format = '%.' + str(options.precision) + 'f'
    output = open(outfile, 'w')
    for token in encoded:
        if charfloat_pat.match(token):
            # in python 2.7, we see a character followed by a float literal
            output.write(token[0] + format % float(token[1:])
        elif float_pat.match(token):
            # in python 2.6, we see a simple float literal
            output.write(format % float(token))
        else:
            output.write(token)
```